In [1]:
import numpy

In [175]:
def distance( x, c ):
    
    
    d = numpy.sqrt( sum( sum( ( x - c )**2 ) ) )
    
    return d

In [176]:
def kmean( x, k, max_iters ):
    
    centroid = x[ numpy.random.choice( range( len( x ) ), k, replace = False ) ]
    ites = 0
    diff = 1
    while ( diff != 0 ) and ( ites <= max_iters ):

        clusters = [ [] for i in range( len( centroid ) ) ]

        for i in range( len( x ) ):

            dist = []

            for j in range( len( centroid ) ):

                dist.append( distance( centroid[ j ], x[ i ] ) )

            clusters[ numpy.where( dist == numpy.min( dist ) )[ 0 ][ 0 ] ].append( x[ i ] ) 

        centroid_minus = centroid.copy()
        centroid = []

        for i in range( len( clusters ) ):

            centroid.append( numpy.mean( clusters[ i ], axis = 0 ) )

        if ( sum( centroid ) - sum( centroid_minus ) ).all() == 0:
            diff = 0
        ites += 1
        
    std = []
    for i in range( len( clusters ) ):
        std.append( numpy.std( clusters[ i ] ) )
    std = numpy.array( std )

    return centroid, std 

In [177]:
class NeuralNet:
    
    def __init__( self, x, y, num_classes, k, max_iters ):
        
        self.input = x
        self.y = y
        self.num_classes = num_classes
        self.k = k
        self.max_iters = max_iters
    
    def one_hot_encode( self, y, num_classes ):
        
        one_hot_arr = numpy.zeros( ( len( y ), num_classes ) )
        
        for i in range( len( y ) ):
            
            one_hot_arr[ i ][ int( y[ i ] ) ] = 1
            
        return one_hot_arr
    
    def rbf( self, x, centroids, std ):
        
        dist = distance( x, centroids )
        rbf = numpy.exp( -dist / std**2 )
        
        return rbf
    
    def RBF( self, x, centroids, std ):
    
        RBF = []
        for i in range( len( x ) ):
            
            RBF.append( [ self.rbf( x[ i ], c, s ) for ( c, s ) in zip( centroids, std ) ] )
                
        return numpy.array( RBF )
    
    def train( self ):
        
        self.centroids, self.std = kmean( self.input, self.k, self.max_iters )
        RBF = self.RBF( self.input, self.centroids, self.std )
        self.weights = numpy.linalg.pinv( RBF.T @ RBF ) @ RBF.T @ self.one_hot_encode( self.y, self.num_classes )
        
    def test( self, test_x, test_y ):
        
        RBF = self.RBF( test_x, self.centroids, self.std )
        self.predict = RBF @ self.weights
        self.pred = numpy.array( [ numpy.argmax( x ) for x in self.predict ] )
        self.diff = self.pred - test_y

        print( 'Accuracy: ', len( numpy.where( self.diff == 0 )[ 0 ] ) / len( self.diff ) )
        

In [178]:
def load_data(path):
    with numpy.load(path) as f:
        x_train, y_train = f['x_train'], f['y_train']
        x_test, y_test = f['x_test'], f['y_test']
        return (x_train, y_train), (x_test, y_test)

( x_train, y_train ), ( x_test, y_test ) = load_data( 'mnist.npz' )

train_y = y_train[ :1000 ]
train_x = x_train[ :1000 ]

test_y = y_test[ 0:500 ]
test_x = x_test[ 0:500 ]

In [179]:
num_classes = 10
k = 100
max_iters = 100

In [180]:
brain = NeuralNet( train_x, train_y, num_classes, k, max_iters )
brain.train()

In [181]:
brain.test( test_x, test_y )

Accuracy:  0.838


In [182]:
test_y

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4,
       6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3,
       6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4,
       8, 7, 3, 9, 7, 4, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6,
       5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5,
       1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7,
       1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 3, 1, 6,
       4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0, 3, 6, 5, 5,
       7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1, 5, 9,
       8, 7, 2, 3, 0, 4, 4, 2, 4, 1, 9, 5, 7, 7, 2, 8, 2, 6, 8, 5, 7, 7,
       9, 1, 8, 1, 8, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2,
       6, 4, 1, 5, 8, 2, 9, 2, 0, 4, 0, 0, 2, 8, 4,

In [183]:
brain.pred

array([7, 2, 1, 0, 4, 1, 4, 9, 4, 9, 0, 4, 9, 0, 1, 5, 4, 7, 3, 4, 9, 6,
       2, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       5, 5, 5, 2, 4, 4, 6, 3, 4, 5, 4, 0, 4, 1, 9, 5, 7, 8, 9, 9, 7, 9,
       4, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 7, 9, 7, 9, 6, 2, 7, 8, 4, 7, 5,
       6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 2, 0, 5, 4, 9, 9, 2, 1, 9, 4,
       8, 1, 3, 9, 7, 4, 9, 4, 9, 8, 5, 4, 7, 6, 4, 9, 0, 5, 8, 5, 6, 6,
       5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 9, 9, 9, 4, 5,
       1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 9, 9, 7, 2, 3, 2, 7,
       1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 4, 0, 5, 1, 6,
       4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 7, 9, 3, 9, 0, 3, 4, 7, 5,
       7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 5, 3, 8, 7, 7, 9, 2, 2, 4, 1, 5, 8,
       8, 4, 2, 6, 0, 6, 4, 2, 9, 1, 9, 5, 7, 7, 2, 8, 2, 0, 8, 1, 7, 7,
       9, 1, 8, 1, 4, 0, 3, 0, 1, 9, 4, 4, 1, 8, 2, 1, 2, 9, 2, 5, 9, 2,
       6, 4, 1, 5, 3, 2, 9, 2, 0, 4, 0, 0, 2, 8, 6,

In [172]:
# import pandas
# test_x = pandas.read_csv( '500_Person_Gender_Height_Weight_Index.csv' )
# test_x = test_x.replace( 'Male', 0 )
# test_x = test_x.replace( 'Female', 1 )
# del test_x['Index']
# test_y = test_x.iloc[ :, 0 ]
# del test_x[ 'Gender' ]
# train_x = test_x.to_numpy( dtype = int )
# test_y = test_y.to_numpy( dtype = int )
# train_y = numpy.empty( ( 500, 1) )
# for i in range( len( train_y ) ):
#     train_y[ i ] = [ test_y[ i ] ]

# test_x = train_x[ 250: ]
# train_x = train_x[ :250 ]
# test_y = train_y[ 250: ]
# train_y = train_y[ :250 ]

In [185]:
# NN = NeuralNet( train_x, train_y, num_classes, k, max_iters )
# NN.train()

In [184]:
# NN.test( test_x, test_y )
# accuracy = len( numpy.where( NN.diff[ 0 ] == 0 )[ 0 ] ) / len( test_x )
# print( 'Real Accuracy =', accuracy )

In [ ]:
# from matplotlib import pyplot
# %matplotlib inline
# for i in range( len( NN.cent ) ):
#     pyplot.scatter( NN.cent[ i ][ 0 ][ 0 ], NN.cent[ i ][ 0 ][ 1 ], color = 'black' )
#     pyplot.scatter( NN.cent[ i ][ 1 ][ 0 ], NN.cent[ i ][ 1 ][ 1 ], color = 'red' )
# pyplot.scatter( train_x[ :, 0 ], train_x[ :, 1 ], color = 'green' );